## Setup libraries

In [7]:
from math import sqrt
import numpy as np
import pandas as pd
from pandasql import sqldf
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping

# Setup tables

In [8]:
Cremona = read_csv("RisultatiCremona.csv")
Cremona = Cremona.loc[:, ["Date", "Ammonia", "PM10", "PM25",
                        "Wind_speed", "Wind_direction", "Temperature", "Rainfall"]]

print(Cremona.shape)
Cremona.head()



(2548, 8)


,Date,Ammonia,PM10,PM25,Wind_speed,Wind_direction,Temperature,Rainfall
0,2014-01-01,2.5,73.0,66.0,NaN,NaN,2.233333,0.0
1,2014-01-02,4.5,51.0,NaN,NaN,NaN,3.658333,0.6
2,2014-01-03,1.6,47.0,46.0,NaN,NaN,4.029167,1.0
3,2014-01-04,1.1,42.0,38.0,NaN,NaN,4.691667,16.6
4,2014-01-05,0.7,21.0,21.0,NaN,NaN,5.412500,19.2


In [9]:
#Primo quadrante
Q1 = Cremona.loc[((Cremona.Wind_direction > 0) &
                 (Cremona.Wind_direction < 90)), :]

a = Q1.shape[0]
arr1 = []
arr1 = [1 for i in range(a)]

del Q1['Wind_direction']
Q1["Wind_direction"] = arr1
columns_titles = ["index", "Date", "Ammonia", "PM10", "PM25",
                  "Wind_speed",  "Temperature", "Rainfall", "Wind_direction" ]
                  
Q1 = Q1.reindex(columns=columns_titles)
Q1 = Q1.drop("index", axis = 1)

#Secondo quadrante
Q2 = Cremona.loc[((Cremona.Wind_direction > 90) &
                 (Cremona.Wind_direction < 180)), :]
a = Q2.shape[0]
arr2 = []
arr2 = [2 for i in range(a)]

del Q2['Wind_direction']
Q2["Wind_direction"] = arr2

#Terzo quadrante
Q3 = Cremona.loc[((Cremona.Wind_direction > 180) &
                 (Cremona.Wind_direction < 270)), :]

a = Q3.shape[0]
arr3 = []
arr3 = [3 for i in range(a)]

del Q3['Wind_direction']
Q3["Wind_direction"] = arr3

#Quarto quadrante
Q4 = Cremona.loc[((Cremona.Wind_direction > 270) &
                 (Cremona.Wind_direction < 360)), :]

a = Q4.shape[0]
arr4 = []
arr4 = [4 for i in range(a)]

del Q4['Wind_direction']
Q4["Wind_direction"] = arr4

Cremona = sqldf('''
SELECT *
FROM Q1
UNION
SELECT *
FROM Q2
UNION
SELECT *
FROM Q3
UNION
SELECT *
FROM Q4
''')
#del Cremona['Wind_direction']
Cremona.rename(columns={'Wind_direction': 'Quadrante'}, inplace=True)

Cremona

/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_9349/2184098774.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q1["Wind_direction"] = arr1
/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_9349/2184098774.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q2["Wind_direction"] = arr2
/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_9349/2184098774.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,Date,Ammonia,PM10,PM25,Wind_speed,Temperature,Rainfall,Quadrante
0,2014-02-12,6.5,54.0,44.0,0.479634,8.004167,0.0,2
1,2014-02-13,8.3,52.0,43.0,1.035337,5.375000,0.6,1
2,2014-02-14,7.7,43.0,36.0,1.061836,7.508333,0.0,4
3,2014-02-15,9.6,41.0,35.0,0.452648,6.404167,0.0,4
4,2014-02-16,7.4,67.0,59.0,0.771591,7.554167,0.2,4
...,...,...,...,...,...,...,...,...
2465,2020-12-27,7.3,35.0,31.0,0.045293,0.952083,0.0,1
2466,2020-12-28,6.7,NaN,20.0,0.212185,0.277465,0.0,4
2467,2020-12-29,6.9,46.0,45.0,0.451607,-0.663889,0.0,4
2468,2020-12-30,7.3,62.0,60.0,0.634008,1.771528,0.0,3


In [10]:
Cremona

,Date,Ammonia,PM10,PM25,Wind_speed,Temperature,Rainfall,Quadrante
0,2014-02-12,6.5,54.0,44.0,0.479634,8.004167,0.0,2
1,2014-02-13,8.3,52.0,43.0,1.035337,5.375000,0.6,1
2,2014-02-14,7.7,43.0,36.0,1.061836,7.508333,0.0,4
3,2014-02-15,9.6,41.0,35.0,0.452648,6.404167,0.0,4
4,2014-02-16,7.4,67.0,59.0,0.771591,7.554167,0.2,4
...,...,...,...,...,...,...,...,...
2465,2020-12-27,7.3,35.0,31.0,0.045293,0.952083,0.0,1
2466,2020-12-28,6.7,NaN,20.0,0.212185,0.277465,0.0,4
2467,2020-12-29,6.9,46.0,45.0,0.451607,-0.663889,0.0,4
2468,2020-12-30,7.3,62.0,60.0,0.634008,1.771528,0.0,3


In [11]:
Cremona.to_csv(
    '/Users/marcovinciguerra/Github/GitTesi/Python/Cremona/QuadrantiCremona/QuadrantiCremona.csv')

